In [ ]:
!pip install datasets

In [ ]:
!pip install gensim
!pip install scipy

from gensim.models import KeyedVectors
from scipy.stats import spearmanr
from datasets import load_dataset

ds = load_dataset("PiC/phrase_similarity")

# Load pretrained embeddings, e.g., GloVe
glove_path = "glove.6B.300d.txt"
model = KeyedVectors.load_word2vec_format(glove_path, binary=False,unicode_errors='ignore')

def evaluate_word_similarity(word_pairs, human_scores):
    predicted_scores = []
    for w1, w2 in word_pairs:
        if w1 in model.key_to_index and w2 in model.key_to_index:
            similarity = model.similarity(w1, w2)
        else:
            similarity = 0  # Handle OOV words
        predicted_scores.append(similarity)

    spearman_corr, _ = spearmanr(human_scores, predicted_scores)
    return spearman_corr

# Example usage
word_pairs = [('bad', 'insane'), ('hard', 'difficulty')]
human_scores = [0.8, 0.6]
print("Spearman Correlation:", evaluate_word_similarity(word_pairs, human_scores))


Downloading builder script: 100%|██████████| 6.38k/6.38k [00:00<00:00, 3.90MB/s]
Downloading metadata: 100%|██████████| 4.04k/4.04k [00:00<00:00, 9.23MB/s]
Downloading readme: 100%|██████████| 7.49k/7.49k [00:00<00:00, 4.55MB/s]
Downloading data files: 100%|██████████| 2/2 [00:00<00:00, 13.1MB/s]
Downloading data: 100%|██████████| 1.86M/1.86M [00:00<00:00, 4.62MB/s]
Downloading data: 100%|██████████| 1.85M/1.85M [00:00<00:00, 3.85MB/s]
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 293it/s]
Generating test split: 100%|██████████| 250/250 [00:00<00:00, 1521/s]
Generating validation split: 100%|██████████| 250/250 [00:00<00:00, 1292/s]
Generating train split: 100%|██████████| 250/250 [00:00<00:00, 1230/s]
loading projection weights from glove.6B.300d.txt
Loading vectors from glove.6B.300d.txt

# *Spearman Correlation: 1.0*

In [ ]:

from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_contextual_embedding(word, sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    word_index = sentence.split().index(word)
    return last_hidden_states[0][word_index].detach().numpy()

# Example usage
sentence = "The bank is on the river"
word = "bank"
embedding = get_contextual_embedding(word, sentence)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_contextual_embedding(word, sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    word_index = sentence.split().index(word)
    return last_hidden_states[0][word_index].detach().numpy()

def calculate_accuracy(predictions, labels):
    correct = 0
    total = len(predictions)
    for pred, label in zip(predictions, labels):
        if pred == label:
            correct += 1
    accuracy = correct / total
    return accuracy

def calculate_similarity(embedding1, embedding2):
    # Reshape embeddings if necessary (ensure they are 2D arrays)
    embedding1 = np.reshape(embedding1, (1, -1))
    embedding2 = np.reshape(embedding2, (1, -1))
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]

# Example usage
sentence1 = "The bank is on the river"
sentence2 = "I deposited money in the bank"
word = "bank"

# Get embeddings
embedding1 = get_contextual_embedding(word, sentence1)
embedding2 = get_contextual_embedding(word, sentence2)

# Calculate similarity score
similarity_score = calculate_similarity(embedding1, embedding2)
print(f"Similarity Score between '{sentence1}' and '{sentence2}': {similarity_score}")

# Assuming labels are known
label1 = "location"
label2 = "action"

# Example accuracy calculation
predictions = ["location", "action"]  # Replace with actual predictions
labels = [label1, label2]
accuracy = calculate_accuracy(predictions, labels)
print(f"Accuracy: {accuracy}")


Similarity Score between 'The bank is on the river' and 'I deposited money in the bank': 0.5505858659744263
Accuracy: 1.0


Similarity Score between 'The bank is on the river' and 'I deposited money in the bank': 0.5505858659744263
## Accuracy: 1.0